In [1]:
from utils import (
    download_file,
    parsear_archivo,
    random_path,
    calcular_distancias,
    cumple_capacidad,
    dist
)
from tqdm.notebook import tqdm as tqdm
import math

# Descargo y parseo datos

In [2]:
download_file("https://modelosuno.okapii.com/Problemas/problema2022/problema_dos.txt", "problema_dos.txt", override=True)

0.00iB [00:00, ?iB/s]

In [3]:
CAPACIDAD, DIMENSION, DEMANDAS, TIPO_ARISTA, COORDS = parsear_archivo("problema_dos.txt")

In [4]:
DATA = [
    (k, v, COORDS[k]["x"], COORDS[k]["y"]) for k,v in DEMANDAS.items()
]
COORDS_S = {
    k: (v["x"], v["y"]) for k, v in COORDS.items()
}

# Armo recorridos

In [5]:
class LoopError(Exception):
    pass

In [11]:
import multiprocessing as mp

def generar_camino(initial, demandas, data, dimension, coords_s, capacidad):
    try:
        if demandas[initial] < 0:
            return None
        seq = [initial]
        seqset = set(seq)
        capacities = [demandas[initial]]

        for _ in range(dimension - 1):
            # while len(seq) < DIMENSION:
            last = seq[-1]
            last_capacity = capacities[-1]
            x0, y0 = coords_s[last]

            best_candidate = None
            best_distance = math.inf

            for k, dem, x1, y1 in data:
                if k in seqset:
                    continue
                if not 0 <= last_capacity + dem <= capacidad:
                    continue
                d = (x0 - x1) ** 2 + (y0 - y1) ** 2
                if d < best_distance:
                    best_candidate = k
                    best_distance = d

            if best_candidate is None:
                raise LoopError

            seq.append(best_candidate)
            seqset.add(best_candidate)
            capacities.append(last_capacity + demandas[best_candidate])

        return seq
    except LoopError:
        return None

import functools

initial_candidates = []
ncandidates = 1000

for i in range(1, DIMENSION+1):
    if len(initial_candidates) >= ncandidates:
        break
    if DEMANDAS[i] >= 0:
        initial_candidates.append(i)

initial_candidates = tuple(initial_candidates)

In [12]:
costos = []
caminos = []

with mp.Pool(24) as p, tqdm(total=len(initial_candidates)) as bar:
    f = functools.partial(
        generar_camino,
        demandas=DEMANDAS,
        data=DATA,
        dimension=DIMENSION,
        coords_s=COORDS_S,
        capacidad=CAPACIDAD
    )
    rs = p.imap(f, initial_candidates)
    for r in rs:
        bar.update(1)
        if r is not None:
            caminos.append(r)
            costos.append(calcular_distancias(r, COORDS))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
idx_minimo = costos.index(min(costos))
camino_minimo = caminos[idx_minimo]

print(idx_minimo)
print(cumple_capacidad(camino_minimo, DEMANDAS, CAPACIDAD))
print(calcular_distancias(camino_minimo, COORDS))

159
True
921761.7640533583


In [14]:
camino_minimo_d = [DATA[x-1] for x in camino_minimo]

In [15]:
def calcular_distancias_l(l):
    total = 0
    _, _, x0, y0 = l[0]
    for k1, dem1, x1, y1 in l[1:]:
        total += math.sqrt((x0 - x1) ** 2 + (y0 - y1) ** 2)
        x0, y0 = x1, y1
    _, _, xo, yo = l[0]
    _, _, xf, yf = l[-1]
    total += math.sqrt((xo - xf) ** 2 + (yo - yf) ** 2)
    return total

def cumple_capacidad_l(l, cap):
    capacidad_actual = 0
    for _, d, _, _ in l:
        capacidad_actual += d
        if capacidad_actual < 0 or capacidad_actual > cap:
            return False
    return True

In [16]:
%%timeit
calcular_distancias_l(camino_minimo_d)

6.34 ms ± 74.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
cumple_capacidad_l(camino_minimo_d, CAPACIDAD)

1.7 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
import time

path = camino_minimo_d

improvement = True
best_path = path
best_cost = calcular_distancias_l(best_path)
capacity_ok = cumple_capacidad_l(best_path, CAPACIDAD)

start = time.time()

with tqdm() as bar:
    while improvement:
        if time.time() - start > 60 * 60:
            break
        improvement = False
        for i in range(len(best_path) - 1):
            bar.set_description(f"Empiezo desde {i}")
            for k in range(i + 1, len(best_path)):
                bar.update()
                new_path = best_path[0:i]
                new_path.extend(reversed(best_path[i:k + 1]))
                new_path.extend(best_path[k + 1:])

                cumple_cap = cumple_capacidad_l(new_path, CAPACIDAD)
                if not cumple_cap:
                    continue

                new_cost = calcular_distancias_l(new_path)

                if new_cost >= best_cost:
                    continue
                else:
                    print(f"Improved from {best_cost} to {new_cost}")
                    best_cost = new_cost
                    best_path = new_path
                    improvement = True
                    break  # improvement found, return to the top of the while loop
            if improvement:
                break

0it [00:00, ?it/s]

Improved from 921761.7640533583 to 921724.9566062796
Improved from 921724.9566062796 to 921688.0024423377
Improved from 921688.0024423377 to 921687.6275443455
Improved from 921687.6275443455 to 921686.6518935711
Improved from 921686.6518935711 to 921661.3107551212
Improved from 921661.3107551212 to 921656.051926259
Improved from 921656.051926259 to 921642.791620995
Improved from 921642.791620995 to 921165.876721186
Improved from 921165.876721186 to 921063.446193951
Improved from 921063.446193951 to 920983.7658409359
Improved from 920983.7658409359 to 920980.5667068079
Improved from 920980.5667068079 to 920951.4151889979
Improved from 920951.4151889979 to 920941.4035315282
Improved from 920941.4035315282 to 920939.7735128897
Improved from 920939.7735128897 to 920939.1309590704
Improved from 920939.1309590704 to 920933.3796992894
Improved from 920933.3796992894 to 920931.6665935831
Improved from 920931.6665935831 to 920912.3237446402
Improved from 920912.3237446402 to 920903.5104622652
I

In [19]:
cumple_capacidad_l(best_path, CAPACIDAD)

True

In [20]:
calcular_distancias_l(best_path)

920848.5037280435

In [21]:
seq_str = " ".join(map(str, [x[0] for x in best_path]))

In [22]:
with open("entrega2_greedy_plus_2opt.txt", "w") as f:
    f.write(seq_str)